In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [5]:
test_data = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [7]:
X = train_data.drop(columns=['FloodProbability'])
y = train_data['FloodProbability']

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_data_scaled = scaler.transform(test_data)

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor()
}

best_model = None
best_rmse = float('inf')

# Train and evaluate models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_scaled, y_train)
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    val_rmse = mean_squared_error(y_val, y_pred_val, squared=False)
    
    print(f"Validation RMSE: {val_rmse:.2f}")
    print()
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_model = model

# Make predictions on test data using the best model and save to CSV
print(f"Making predictions using the best model...")
predictions = best_model.predict(test_data_scaled)
submission = pd.DataFrame({'id': test_data['id'], 'FloodProbability': predictions})
submission.to_csv("best_model_submission.csv", index=False)
print(f"Predictions saved for the best model")

Training Linear Regression...
Validation RMSE: 0.02

Training Decision Tree...
Validation RMSE: 0.05

Training Gradient Boosting...
Validation RMSE: 0.03

Training Support Vector Machine...
Validation RMSE: 0.03

Training XGBoost...
Validation RMSE: 0.02

Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.152351 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 624
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 21
[LightGBM] [Info] Start training from score 0.504480
Validation RMSE: 0.02

Training CatBoost...
Learning rate set to 0.119817
0:	learn: 0.0501810	total: 189ms	remaining: 3m 9s
1:	learn: 0.0493271	total: 310ms	remaining: 2m 34s
2:	learn: 0.0484614	total: 432ms	remaining: 2m 23s
3:	learn: 0.0476903	total: 546ms	remaining: 2m 16s
4:	learn: 0.0469429	total: 656ms	remaining: 2m 10s
5:	learn: 0.0462105	total: 768ms	remaining: 2m 7s
6:	

In [ ]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

2024-05-05 04:05:20.815345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 04:05:20.815447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 04:05:20.951312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_val_scaled, y_val), verbose=1)

# Evaluate the model
train_loss = model.evaluate(X_train_scaled, y_train, verbose=0)
val_loss = model.evaluate(X_val_scaled, y_val, verbose=0)
print(f"Train loss: {train_loss:.4f}")
print(f"Validation loss: {val_loss:.4f}")

# Make predictions on test data
predictions = model.predict(test_data_scaled)

# Save predictions to CSV
submission = pd.DataFrame({'id': test_data['id'], 'FloodProbability': predictions.flatten()})
submission.to_csv("deep_learning_submission.csv", index=False)
print("Predictions saved for the deep learning model")